# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [22]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email, instagram, or PDF/document download links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'}]}

In [9]:
select_relevant_links("https://www.bosch-softwaretechnologies.com/en/")

{'links': [{'type': 'about page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/'},
  {'type': ' manifesto page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/our-manifesto/'},
  {'type': 'partnerships page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/partnership-and-alliances/'},
  {'type': 'awards page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/bgsw-awards/'},
  {'type': 'team page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/our-people/'},
  {'type': 'partnership program page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/dna-nxt-start-up-alliance-program%E2%80%8B/'},
  {'type': 'csr page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/corporate-social-responsibility/'},
  {'type': 'csr projects',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-c

In [15]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [11]:
select_relevant_links("https://www.bosch-softwaretechnologies.com/en/")

Selecting relevant links for https://www.bosch-softwaretechnologies.com/en/ by calling gpt-5-nano
Found 53 relevant links


{'links': [{'type': 'about page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/'},
  {'type': 'manifesto page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/our-manifesto/'},
  {'type': 'partnerships page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/partnership-and-alliances/'},
  {'type': 'awards page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/bgsw-awards/'},
  {'type': 'team page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/our-people/'},
  {'type': 'startup alliance program page',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/about-us/dna-nxt-start-up-alliance-program%E2%80%8B/'},
  {'type': 'csr overview',
   'url': 'https://www.bosch-softwaretechnologies.com/en/our-company/corporate-social-responsibility/'},
  {'type': 'csr projects',
   'url': 'https://www.bosch-softwaretechnologies.com/

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://www.bosch-softwaretechnologies.com/en/"))

Selecting relevant links for https://www.bosch-softwaretechnologies.com/en/ by calling gpt-5-nano
Found 68 relevant links
## Landing Page:

Bosch Global Software Technologies | Bosch Global Software Technologies PVT LTD

Skip to main content
Our Company
Main Navigation
Our Company
About Us
Main Navigation
Our Company Overview
About Us
Overview
Our Manifesto
Partnership and Alliances
BGSW Awards
Our People
DNA Nxt - Start-up Alliance Program​
Corporate Social Responsibility
Main Navigation
Our Company Overview
Corporate Social Responsibility
Overview
CSR Projects
CSR Events
News and Press Release
NGO Partners
Team
Videos
Press Releases
Main Navigation
Our Company Overview
Press Releases
Overview
2019
2020
2021
2022
2023
Alumni
Products and Solutions
Main Navigation
Products and Solutions
Overview
Products & Solutions
Main Navigation
Products and Solutions Overview
Products & Solutions
Overview
AIShield
Autotrace
Evoco
Connected Mobility Solutions
Phantom
SVENTA
Trac 360
Vivalution
Vivar

In [28]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, hilarious, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Bosch Global Software Technologies", "https://www.bosch-softwaretechnologies.com/en/")

In [35]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    return result

In [24]:
create_brochure("Bosch Global Software Technologies", "https://www.bosch-softwaretechnologies.com/en/")

Selecting relevant links for https://www.bosch-softwaretechnologies.com/en/ by calling gpt-5-nano
Found 31 relevant links


# Bosch Global Software Technologies Brochure

---

## Who We Are

Bosch Global Software Technologies Pvt Ltd (BGSW) is a forefront provider of advanced software solutions and engineering services, committed to pioneering innovation across diverse industries. As part of the Bosch Group, we leverage decades of expertise in technology to create impactful software products and services that enhance mobility, healthcare, manufacturing, and retail sectors worldwide.

---

## Our Manifesto

At the heart of BGSW lies a clear manifesto: to drive digital transformation through cutting-edge AI, IoT, and smart technology solutions while nurturing a culture of creativity, collaboration, and continuous learning. We believe in pushing the boundaries of software innovation, empowering our clients to thrive in a connected and fast-evolving world.

---

## Our Solutions & Services

We offer a broad spectrum of products and services tailored to meet dynamic industry needs:

### Products & Solutions
- **AIShield**: Advanced AI-based security systems
- **Autotrace**: Intelligent tracing and diagnostics tools
- **Evoco**: Software for mobility ecosystem management
- **Connected Mobility Solutions**: Seamless integration for smart transportation
- **Phantom / SVENTA / Trac 360**: Cutting-edge analytics and testing platforms
- **Vivalution & Vivaray**: Next-gen healthcare and imaging solutions

### Engineering Services
- AIoT-led Digital Transformation
- Smart Product Design & Development (Design Studio)
- Healthcare Engineering & IoMT Solutions
- Human-Machine Interface (HMI) Engineering
- Intelligent Lifecycle Management
- Testing & Validation Frameworks
- IoT Garage: Rapid prototyping and solution incubator

### Enterprise Services
- AI & Big Data Analytics
- Cloud and IoT Integration
- Cognitive Automation for efficiency
- Customer Experience Enhancement
- Energy Platform Solutions
- Manufacturing Excellence & Supply Chain Optimization

---

## Industries We Serve

BGSW drives innovation across multiple verticals including:
- **Mobility**: Autonomous vehicles, connected cars, and smart city mobility
- **Healthcare**: Digital therapeutics, Internet of Medical Things (IoMT), targeted healthcare solutions with Bosch Vivasuite
- **Retail**: Intelligent retail solutions enhancing customer experience
- **Manufacturing**: Smart factories and Industry 4.0 adoption 

---

## Our People and Culture

Our greatest asset is our people — a diverse team of passionate engineers, scientists, and innovators aligned with the Bosch values of quality, sustainability, and ingenuity. Our culture promotes agility, inclusiveness, and collaboration to foster innovation. BGSW believes in investing in employee growth through continuous learning programs, startup alliances (DNA Nxt), and leadership development.

---

## Corporate Social Responsibility (CSR)

BGSW is committed to social impact and sustainable growth. We actively engage in CSR projects and events supporting education, healthcare, and community development, collaborating with multiple NGO partners to create a positive societal influence.

---

## Innovation and Global Presence

BGSW operates globally with key innovation centers including a notable presence in Germany with on-site delivery centers dedicated to R&D and customer-centric innovations. Our multi-market approach enables us to customize solutions to local needs while maintaining global standards.

---

## Careers at Bosch Global Software Technologies

We invite talented professionals, students, and graduates to join us in shaping the future of software innovation. Our career programs offer:
- A vibrant work environment valuing diversity and creativity
- Opportunities to work on cutting-edge AI, IoT, healthcare, and mobility projects
- Structured learning and development paths
- Engaging in meaningful projects with global impact

Explore open positions and learn why Bosch is a preferred employer in the technology domain.

---

## Connect with Us

Explore more about our company, products, solutions, and career opportunities at:

**Website:** [Bosch Global Software Technologies](https://www.bosch-global-softwaretechnologies.com)

---

**Bosch Global Software Technologies** — Innovating today for a smarter, safer tomorrow.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [30]:
stream_brochure("Bosch Engineering", "https://www.bosch-engineering.com/")

Selecting relevant links for https://www.bosch-engineering.com/ by calling gpt-5-nano
Found 14 relevant links


# Welcome to Bosch Engineering — The Ultimate Engineering Marvels Factory 🚀⚙️

---

## Who Are We?  
Bosch Engineering is like the Swiss Army knife of the engineering world — versatile, precise, and always ready to save your project day. We’re your ultimate engineering sidekick, crafting cutting-edge solutions onboard your vehicles and offboard systems to keep the world moving smoothly — whether on street, land, air, water or rail! 🚗🚜✈️🛥️🚊  

From passenger cars that purr like kittens, to agricultural machinery that works harder than your morning espresso, we engineer everything with passion, precision, and a pinch of magic.

---

## Our Engineering Wizardry 🧙‍♂️✨

- **On-board & Off-board Engineering:** Because sometimes it's what's inside that counts (like your car's brain).
- **Cybersecurity & Functional Safety:** Keeping your ride safer than a seatbelt and smarter than your phone.
- **Diagnostics:** We find the problem before it knows it has one.
- **Mobility Solutions:** Powering motion, energy, ADAS (that’s Advanced Driver Assistance Systems, in case you were wondering), body & comfort, and connected services — the stuff that makes your vehicle the coolest on the block.
- **Testing & Prototyping:** With an arsenal of test benches, labs, and workshops, we test things till they break (but don’t worry, we fix them).
- **Consulting & Trainings:** Because even the best need a coach.

---

## Where We Bring the Fun — Industries We Rock 🎉

- **Street:** Passenger cars, commercial vehicles, powersports, motorsports — your daily heroes and weekend racers.
- **Land:** Agricultural, construction, robotic machinery — we build the muscle and brains for the heavy lifters.
- **Air:** General aviation that lifts you up and keeps you flying high.
- **Water:** Motorboats and sailing yachts, cruising the waves with engineered elegance.
- **Rail:** Light rail, trams, and railway — because trains should run like clockwork (and be way cooler).

---

## Our Culture: Engineering with a Smile 😄💡

At Bosch Engineering, work isn’t just about nuts and bolts — it’s about dreaming big and engineering bigger. Our culture blends:

- A passion for innovation (because why settle for good when you can have mind-blowing?).
- Teamwork tighter than a well-tuned engine.
- Continuous learning through workshops, competitions, and recruiting events.
- Diversity, respect, and an atmosphere where ideas rev up like turbo engines.

---

## Careers — Join Our Dream Team! 🚀

Whether you’re a seasoned pro, fresh out of college, or still trying to pick the right engineering tools in school, Bosch Engineering has your back:

- **Professionals:** Push the boundaries of your career and jump into world-changing projects.
- **Career Starters & Students:** Hands-on workshops, student competitions, and the chance to learn from engineering legends.
- **Pupils:** Early career guidance because it’s never too early to start dreaming big.

Ready to engineer the future? We’re ready to hand you the toolkit.

---

## Why Bosch Engineering? Because We Make Magic Happen 🔧✨

- Unmatched technical prowess across multiple industries.
- A global presence with locations buzzing with ideas.
- End-to-end services from concept to final testing.
- Passionate experts who love turning wild ideas into reality.

So, if you want to partner with the engineering maestros who conjure smarter, safer, and smoother mobility solutions — give Bosch Engineering a shout!

---

**Bosch Engineering: Engineering Your World, One Brilliant Idea at a Time.**  
(And yes, we make cars smarter, veggies grow better, and your daily commute way cooler.)

*Contact us — if you dare to dream beyond the ordinary.* 

---

# Bosch Engineering  
*Where Engineering Meets Awe.*

In [29]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Bosch Global Software Technologies", "https://www.bosch-softwaretechnologies.com/en/")

Selecting relevant links for https://www.bosch-softwaretechnologies.com/en/ by calling gpt-5-nano
Found 61 relevant links


# Welcome to the Witty World of Bosch Global Software Technologies (BGSW)!  

*Where Innovation Meets Fun, and Tech Talks in Tongues You Didn't Know Existed*

---

## Who Are We?  
Imagine a place where code is brewed with passion, technology dances with creativity, and software solutions are crafted like gourmet meals. That, in a nutshell, is Bosch Global Software Technologies PVT LTD! We are the digital heartbeat of Bosch, spinning smart, savvy software that powers everything from your car to your healthcare (yes, we make devices smarter and doctors happier).

---

## Our Superpowers (a.k.a Products & Solutions)  
Meet the Avengers of the software universe:  
- **AIShield** – The superhero guarding your data like a secret agent on caffeine.  
- **Autotrace** – For those who want their vehicles smart enough to find their own way home (no backseat drivers needed!).  
- **Evoco & Connected Mobility Solutions** – Driving innovation right into your driveway.  
- **Phantom, SVENTA, Trac 360, Vivalution & Vivaray** – Various dazzling tools making everything smarter, slicker, and yes, cooler.  

Got a problem? We have a digital wizard ready to cast the perfect spell!

---

## Our Culture: Where Nerds Thrive & Ideas Party  
Work here is not just about sitting behind screens – it’s about teaming up, dreaming big, and occasionally debating if AI will ever take over the coffee machine. We foster a diverse, inclusive culture where engineers, thinkers, and doers unite to create magic. There's also a "DNA Nxt - Start-up Alliance Program" because why not sprinkle some startup spice into our big company stew?

We believe in:  
- **Innovation with a conscience** – Our Corporate Social Responsibility projects aren’t just for show; giving back is part of our DNA.  
- **Learning & Growing** – Students, graduates, and career ninjas, we have boot camps, design studios, and IoT garages where you can roll up your sleeves and geek out.  
- **Recognition & Fun** – Awards? We have those. Celebrations? Absolutely. We know software is serious, but ourselves? Not so much.  

---

## Who Loves Us?  
From automotive giants and healthcare heroes to manufacturing marvels and retail wizards—our clients are everywhere, proving that BGSW isn’t just a company; it’s a movement. Whether it’s crafting seamless customer experiences or automating supply chains with AI and Big Data, we plug into the pulse of various industries.

---

## Careers: Join The Software Circus!  
Why just work when you can **come alive**? If you’re a tech enthusiast who loves challenges, quirky colleagues, and impact at scale, slide into our hiring portal. Positions range from AI maestros and cloud wranglers to testing gurus and digital therapists (yes, that’s a thing here).

Highlights for careers at Bosch Global Software Technologies:  
- Meaningful projects across the globe including Germany and beyond  
- Culture that values your brainwaves and your memes  
- Career development that includes students and graduates stepping confidently into the tech arena  

If you want to add "solving the world’s problems with software" to your CV, your dream job is just a click away.

---

## Final Words: The Bosch Manifesto in a Nutshell  
We’re not just coding software; we’re coding the future! Commitment, collaboration, and a pinch of wit fuel us. Expect bold ideas, fearless innovation, and a relentless pursuit of excellence with a human touch.

---

# So, Ready to Make Magic with BGSW?  
Whether you want to invest in the next big thing, work where your geek flag flies high, or partner with innovators who think different (and binary), Bosch Global Software Technologies is your go-to powerhouse.

**Bosch Global Software Technologies** — Because the future is software-written and laughter-powered.

---

*Disclaimer: No coffee machines were harmed in the making of this brochure (yet).*

In [31]:
translate_system_prompt = """
You are a translation assistant that translates English text into Hindi, without losing the meaning or context.
"""
def get_translate_user_prompt(company_name, url, language="Hindi"):
    user_prompt = f"""
    Please translate the following English text into {language}:
    """
    user_prompt += create_brochure(company_name, url)
    return user_prompt



In [32]:
def translate_brochure(company_name, url, language="Hindi"):
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": translate_system_prompt},
            {"role": "user", "content": get_translate_user_prompt(company_name, url, language)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [39]:
translate_brochure("Bosch Engineering", "https://www.bosch-engineering.com/", language="Japanese with proper Furigana for Kanjis")

Selecting relevant links for https://www.bosch-engineering.com/ by calling gpt-5-nano
Found 33 relevant links


# ボッシュ・エンジニアリング：現代（げんだい）モビリティの舞台裏（ぶたいうら）の驚異（きょうい） 🚗⚙️

## 私たちとは — エンジニアリングの魔法使（まほうつか）いたち 🧙‍♂️🔧

ボッシュ・エンジニアリングでは、単（たん）にエンジニアリングをするのではなく、正確（せいかく）さと情熱（じょうねつ）、そしてほんの少しの魔法（まほう）をもって未来（みらい）のモビリティを*創造（そうぞう）*します。ボッシュの長年（ながねん）の輝（かがや）きから生（う）まれた私たちは、洗練（せんれん）された乗用車（じょうようしゃ）、活気（かっき）ある商用車（しょうようしゃ）、高速（こうそく）モータースポーツ、そして明日（あす）の空や水上（すいじょう）での驚異（きょうい）の裏側（うらがわ）の秘密（ひみつ）のソースです。

車載（しゃさい）システムはあなたのスマートフォンより賢（かしこ）く、オフボード技術（ぎじゅつ）は舞台裏（ぶたいうら）ですべてを動（うご）かし、サイバーセキュリティはデジタルの要塞（ようさい）のようにあなたの乗り物（のりもの）を守（まも）ります — 私たちはすべてをカバーします。安全性（あんぜんせい）？診断（しんだん）？動（うご）き？エネルギー？イエス、すべてチェック済（ちぇっくず）みです。

## 私たちのスーパーパワー ⚡🛠️

- **エンジニアリングサービス：** 車載（しゃさい）、非車載（ひしゃさい）、安全性（あんぜんせい）、診断（しんだん）、サイバーセキュリティ — あなたが名付（なづ）けるもの、私たちはスイス製時計（どけい）のような精密性（せいみつせい）で設計（せっけい）します。
- **モビリティソリューションズ：** 動（うご）き、エネルギー、高度運転支援システム（ADAS）、ボディと快適（かいてき）性（せい）（快適な乗り心地が誰（だれ）だって好きじゃないはずがない）、コネクテッドサービス。
- **テスト＆プロトタイピング：** 試験（しけん）ベンチ、ラボラトリー、プロトタイプ製造（せいぞう） — 私たちは壊（こわ）してからあなたに安心（あんしん）をお届けします。
- **コンサルティング＆トレーニング：** 運用（うんよう）優秀性（ゆうしゅうせい）、変革（へんかく）、品質管理（ひんしつかんり）、エンジニアリングスキル向上（こうじょう）ワークショップ。

## 私たちが魅了（みりょう）する産業（さんぎょう） 🌍🚀

- **ストリート：** 乗用車（じょうようしゃ）、商用車（しょうようしゃ）、パワースポーツ、モータースポーツ（はい、私たちはレースカーをより速（はや）く、より賢（かしこ）くします）。
- **陸（りく）：** 農業機械（のうぎょうきかい）、建設機械（けんせつきかい）、イントラロジスティクス、ロボティクス、市営（しえい）および空港車両（くうこうしゃりょう）。
- **空（くう）＆水上（すいじょう）：** 一般航空機（いっぱんこうくうき）、モーターボート、セーリングヨット — 速度（そくど）と安全性（あんぜんせい）は陸上（りくじょう）だけにとどまりません。
- **鉄道（てつどう）：** ライトレール、路面電車（ろめんでんしゃ）、鉄道 — 最新技術（さいしんぎじゅつ）で公共交通（こうきょうこうつう）を正（ただ）しく軌道（きどう）に乗（の）せます。

## 顧客（こきゃく）＆パートナー — 私たちのクールな仲間（なかま） 😎🤝

私たちの存在（そんざい）をほとんど知らない日常（にちじょう）の運転手（うんてんしゅ）から、安全にブレーキをかけるたびに感謝（かんしゃ）されるまで、そして乗り物（のりもの）をスマートで安全、将来（しょうらい）に備（そな）えさせることを依頼（いらい）する大手製造業者（せいぞうぎょうしゃ）まで—ボッシュ・エンジニアリングはモビリティの最先端（さいせんたん）トレンドの静（しず）かなパートナーです。

もしあなたが車を運転（うんてん）し、飛び、航海（こうかい）し、また公共交通機関（こうきょうこうつうきかん）を利用（りよう）するなら、おそらく私たちの仕事はあなたの車輪（しゃりん）、翼（つばさ）、水上機（すいじょうき）の下（した）にあります。

## キャリア — 非凡（ひぼん）なエンジニアのリーグに参加（さんか）しよう 👩‍💻👨‍🔧💥

普段（ふだん）の回路（かいろ）に飽（あ）きたプロのエンジニアですか？ノートに天才的（てんさいてき）な落書（らくが）きをする学生（がくせい）ですか？それとも機械（きかい）を操（あやつ）る夢（ゆめ）を見る生徒（せいと）ですか？ボッシュ・エンジニアリングはあなたを求（もと）めています。

- **プロのキャリア：** 現実（げんじつ）の課題（かだい）に挑（いど）み、未来技術（みらいぎじゅつ）を発明（はつめい）し、次（つぎ）の自動運転車（じどううんてんしゃ）を作（つく）るかも。
- **キャリアスタート：** 実践的（じっせんてき）なプロジェクトと、技術（ぎじゅつ）ジョークも笑（わら）いも大好きなチームと共（とも）にエンジニアリング冒険（ぼうけん）を開始（かいし）。
- **学生（がくせい）＆生徒（せいと）：** 学生コンペティション、ワークショップ、採用イベント — 学んで、競（きそ）い、夢（ゆめ）のキャリアをスタート。

*ここで働（はたら）くことは、イノベーション、協力（きょうりょく）、そして品質（ひんしつ）への健全（けんぜん）なこだわりを大切（たいせつ）にする文化（ぶんか）の一員（いちいん）になることです。あなたは最先端の頭脳（ずのう）とともに働き、アイデアがプロトタイプになり、プロトタイプが伝説（でんせつ）になるラボやワークショップにアクセスできます。*

## なぜボッシュ・エンジニアリング？ それは... 🤔

- 単（たん）に製品（せいひん）を設計（せっけい）するのではなく、**体験（たいけん）**を設計するから。
- 伝統（でんとう）と変革（へんかく）を融合（ゆうごう） — ボッシュの数十年（すうじゅうねん）の遺産（いさん）が最先端（さいせんたん）イノベーションと出会（であ）います。
- 私たちの遊び場（あそびば）は広大（こうだい）です：高速道路（こうそくどうろ）、農地（のうち）、水路（すいろ）、鉄道路線（てつどうろせん）、そして空（そら）さえも。
- 私たちの文化？　ハイテクとハイファンが融合（ゆうごう）し、チームワークと知力（ちりょく）が支配（しはい）する世界（せかい）。
- 壊（こわ）れるまでテストし、壊れなくなるまで改良（かいりょう）する—これを繰（く）り返（かえ）します。

---

**ボッシュ・エンジニアリング：** あなたの乗り物（のりもの）、あなたの旅（たび）、あなたの明日（あす）をパワーアップ — ウインクとレンチ、そしてたっぷりの驚き（おどろき）と共（とも）に。

*シートベルトを締（し）めて、未来はここで設計（せっけい）される。* 🚀

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>